In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from scipy import stats
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


train = pd.read_csv('../input/train.csv')
train.info()

# Any results you write to the current directory are saved as output.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(data=train.corr())
plt.gcf().clear
plt.show()


In [ ]:
train.skew()

In [ ]:
train_d = train.copy()
train_d = pd.get_dummies(train_d)
train_d.info()

In [ ]:
keep_cols = train_d.select_dtypes(include=['number']).columns

train_d = train_d[keep_cols]
train_d.describe()

In [ ]:
train_d = train_d.fillna(train_d.mean())
train_d.head()

In [ ]:
test = pd.read_csv("../input/test.csv")
test.head()

In [ ]:
test_d = test.copy()
test_d = pd.get_dummies(test_d)
test_d.info()

In [ ]:
test_d = test_d.fillna(test_d.mean())

In [ ]:
for col in keep_cols:
    if col not in test_d:
        test_d[col] = 0
test_d.info()

In [ ]:
test_d = test_d[keep_cols]
test_d.info()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
rf_test = RandomForestRegressor(max_depth=30, n_estimators=500, max_features = 100, oob_score=True, random_state=1234)
cv_score = cross_val_score(rf_test, train_d.drop('SalePrice', axis = 1), train_d['SalePrice'], cv = 10, n_jobs = -1)


In [ ]:
print('CV Score is: '+ str(np.mean(cv_score)))

In [ ]:
train_0 = train.copy()
null_index = train_0.LotFrontage.isnull()

In [ ]:
train_0.loc[null_index,'LotFrontage'] = 0

In [ ]:
train_0 = pd.get_dummies(train_0)
keep_cols = train_0.select_dtypes(include=['number']).columns
train_0 = train_0[keep_cols]
train_0 = train_0.fillna(train_0.mean())

In [ ]:
rf_test = RandomForestRegressor(max_depth=30, n_estimators=500, max_features = 100, oob_score=True, random_state=1234)
cv_score = cross_val_score(rf_test, train_0.drop('SalePrice', axis = 1), train_0['SalePrice'], cv = 10, n_jobs = -1)


In [ ]:
print('CV Score is: '+ str(np.mean(cv_score)))

In [ ]:
sns.barplot(data=train,x='Neighborhood',y='LotFrontage', estimator=np.median)
plt.xticks(rotation=90)
plt.show()


In [ ]:
gb_neigh = train['LotFrontage'].groupby(train['Neighborhood'])
for i in gb_neigh:
    print (i)

In [ ]:
train_LFm = train.copy()

In [ ]:
for key,group in gb_neigh:
    # find where we are both simultaneously missing values and where the key exists
    lot_f_nulls_nei = train['LotFrontage'].isnull() & (train['Neighborhood'] == key)
    # fill in those blanks with the median of the key's group object
    train_LFm.loc[lot_f_nulls_nei,'LotFrontage'] = group.median()

In [ ]:
train_LFm = pd.get_dummies(train_LFm)
keep_cols = train_LFm.select_dtypes(include=['number']).columns
train_LFm = train_LFm[keep_cols]

In [ ]:
train_LFm = train_LFm.fillna(train_LFm.mean())

In [ ]:
#rf_test = RandomForestRegressor(max_depth=30, n_estimators=500, max_features = 100, oob_score=True, n_jobs=-1, random_state= 1234)
#cv_score = cross_val_score(rf_test, train_LFm.drop('SalePrice', axis = 1), train_LFm['SalePrice'], cv = 10, n_jobs=-1)

In [ ]:
#print('CV Score is: '+ str(np.mean(cv_score)))

In [ ]:
from xgboost.sklearn import XGBRegressor

In [ ]:
xgb_test = XGBRegressor(learning_rate=0.05,n_estimators=1000,max_depth=3,colsample_bytree=0.4)
cv_score = cross_val_score(xgb_test, train_LFm.drop(['SalePrice','Id'], axis = 1), train_LFm['SalePrice'], cv = 15, n_jobs = -1)

In [ ]:
print('CV Score is: '+ str(np.mean(cv_score)))

In [ ]:
xgb_test.fit(train_LFm.drop('SalePrice',axis = 1),train_LFm['SalePrice'])

In [ ]:
preds = xgb_test.predict(test_d.drop('SalePrice', axis=1))
out_preds = pd.DataFrame()
out_preds['Id'] = test['Id']
out_preds['SalePrice'] = preds
out_preds.to_csv('output.csv', index=False)